In [2]:
import pandas as pd
import pathlib
import os
import requests
from tqdm import tqdm
from merge_datasets import merge_datasets

In [3]:
# Retrieve standardized column names 
STANDARD_COLS = requests.get(
    "https://raw.githubusercontent.com/uva-bi-sdad/sdc.metadata/master/data/column_structure.json"
).json()
STANDARD_COLS

['geoid', 'measure', 'moe', 'value', 'year']

In [4]:
# If more counties exist, only work with the following counties 

COUNTIES_TO_FOCUS = [
    "01073",
    "01117",
    "01007",
    "01125",
    "01043",
    "01009",
    "01115",
    "13057",
    "13117",
    "13135",
    "13247",
    "13151",
    "13113",
    "13089",
    "13121",
    "13097",
    "13067",
    "13063",
]

In [5]:
# Some general utility functions
def perc(a, b):
    return a / b * 100

def export_per_county(export_dir, filenames_to_df_dict):
    '''
    Given an export directory, and a dictionary with filenames to data frames, save the results
    '''
    pbar = tqdm(filenames_to_df_dict)
    for key in pbar:
        export_filepath = os.path.join(export_dir, key.name)
        pbar.set_description("Saving file to: %s" % export_filepath)

        df = filenames_to_df_dict[key]
        # Sanity check that the export data frame makes sense
        assert not any(df["geoid"].isnull())
        df.to_csv(export_filepath, index=False)

In [6]:
# Linking data sources
ookla_data_dir = "../sdc.broadband.ookla/data/distribution"
acs_data_dir = "../sdc.broadband.acs/data/distribution"
bbn_data_dir = "../sdc.broadband.broadbandnow/data/distribution"

assert os.path.isdir(bbn_data_dir) and os.path.isdir(acs_data_dir) and os.path.isdir(ookla_data_dir)

## avg_up_using_devices

In [19]:
files = sorted(pathlib.Path(ookla_data_dir).glob("*.csv.xz"))
df = pd.read_csv(files[0], dtype={'GEOID20':object})
df

,GEOID20,avg_d_mbps,avg_u_mbps,tests,devices,avg_lat_ms,year,q
0,010010208032004,300.460,346.816,2,2,2,2022,1
1,010010208032004,420.402,540.529,1,1,3,2022,1
2,010010208032004,453.762,644.528,4,1,3,2022,1
3,010010208032004,209.799,211.998,2,2,1,2022,1
4,010010208032004,456.482,550.670,3,1,2,2022,1
...,...,...,...,...,...,...,...,...
9215,010010207003002,15.327,9.903,3,2,26,2022,4
9216,010010207003031,440.002,21.072,2,1,37,2022,4
9217,010010207003028,337.272,312.214,10,3,17,2022,4
9218,010010205011028,337.272,312.214,10,3,17,2022,4


In [22]:
# Remove all not nulls but also the specific none-standard values
df = df[(df["avg_u_mbps"] >= 0) & (df["avg_u_mbps"].notnull())].copy()
df = df.rename(columns={"GEOID20": "geoid"})
df

,geoid,avg_d_mbps,avg_u_mbps,tests,devices,avg_lat_ms,year,q
0,010010208032004,300.460,346.816,2,2,2,2022,1
1,010010208032004,420.402,540.529,1,1,3,2022,1
2,010010208032004,453.762,644.528,4,1,3,2022,1
3,010010208032004,209.799,211.998,2,2,1,2022,1
4,010010208032004,456.482,550.670,3,1,2,2022,1
...,...,...,...,...,...,...,...,...
9215,010010207003002,15.327,9.903,3,2,26,2022,4
9216,010010207003031,440.002,21.072,2,1,37,2022,4
9217,010010207003028,337.272,312.214,10,3,17,2022,4
9218,010010205011028,337.272,312.214,10,3,17,2022,4


## Faster implementation?

In [62]:
df['upload_speed'] = df['avg_u_mbps'] * df['devices']
df

,geoid,avg_d_mbps,avg_u_mbps,tests,devices,avg_lat_ms,year,q,upload_speed
0,010010208032004,300.460,346.816,2,2,2,2022,1,693.632
1,010010208032004,420.402,540.529,1,1,3,2022,1,540.529
2,010010208032004,453.762,644.528,4,1,3,2022,1,644.528
3,010010208032004,209.799,211.998,2,2,1,2022,1,423.996
4,010010208032004,456.482,550.670,3,1,2,2022,1,550.670
...,...,...,...,...,...,...,...,...,...
9215,010010207003002,15.327,9.903,3,2,26,2022,4,19.806
9216,010010207003031,440.002,21.072,2,1,37,2022,4,21.072
9217,010010207003028,337.272,312.214,10,3,17,2022,4,936.642
9218,010010205011028,337.272,312.214,10,3,17,2022,4,936.642


In [101]:
avg_u_sum = df.groupby(['geoid','year'])['upload_speed'].agg('sum').to_frame().reset_index()
devices_sum = df.groupby(['geoid','year'])['devices'].agg('sum').to_frame().reset_index()
fdf = pd.merge(avg_u_sum, devices_sum, on=['geoid','year'])
fdf['measure'] = 'avg_up_using_devices'
fdf['value'] = fdf['upload_speed']/fdf['devices']
fdf = fdf.reindex(STANDARD_COLS, axis="columns")
fdf['region_type'] = 'block_group'
fdf

,geoid,measure,moe,value,year,region_type
0,010010201001000,avg_up_using_devices,NaN,11.002167,2022,block_group
1,010010201001001,avg_up_using_devices,NaN,14.373455,2022,block_group
2,010010201001002,avg_up_using_devices,NaN,22.760750,2022,block_group
3,010010201001003,avg_up_using_devices,NaN,99.174238,2022,block_group
4,010010201001004,avg_up_using_devices,NaN,115.386400,2022,block_group
...,...,...,...,...,...,...
1268,010010211004042,avg_up_using_devices,NaN,12.656625,2022,block_group
1269,010010211004043,avg_up_using_devices,NaN,11.617857,2022,block_group
1270,010010211004044,avg_up_using_devices,NaN,14.747615,2022,block_group
1271,010010211004045,avg_up_using_devices,NaN,11.617857,2022,block_group


## Checking

In [68]:
test_df = df[df['geoid'] ==  '010010201001000']
test_df

,geoid,avg_d_mbps,avg_u_mbps,tests,devices,avg_lat_ms,year,q,upload_speed
2152,010010201001000,19.441,2.951,1,1,37,2022,1,2.951
2153,010010201001000,417.402,22.474,1,1,24,2022,1,22.474
2541,010010201001000,84.397,11.279,3,2,30,2022,2,22.558
2542,010010201001000,28.886,7.950,4,1,22,2022,2,7.950
5846,010010201001000,111.138,9.730,7,2,24,2022,3,19.460
5847,010010201001000,156.625,11.673,2,1,24,2022,3,11.673
5848,010010201001000,208.911,11.970,5,1,24,2022,3,11.970
8223,010010201001000,109.299,9.855,1,1,25,2022,4,9.855
8224,010010201001000,25.410,11.492,6,1,23,2022,4,11.492
8225,010010201001000,21.992,11.643,12,1,23,2022,4,11.643


In [69]:
sum(test_df['avg_u_mbps']* test_df['devices']) / sum(test_df['devices'])

11.002166666666668

In [70]:
assert fdf[fdf['geoid'] == '010010201001000']['value'].values[0] == sum(test_df['avg_u_mbps']* test_df['devices']) / sum(test_df['devices'])

## Aggregating to a higher geography level (county: 5 characters, tract: 11 digits)

In [89]:
df

,geoid,avg_d_mbps,avg_u_mbps,tests,devices,avg_lat_ms,year,q,upload_speed,census_tract
0,010010208032004,300.460,346.816,2,2,2,2022,1,693.632,01001020803
1,010010208032004,420.402,540.529,1,1,3,2022,1,540.529,01001020803
2,010010208032004,453.762,644.528,4,1,3,2022,1,644.528,01001020803
3,010010208032004,209.799,211.998,2,2,1,2022,1,423.996,01001020803
4,010010208032004,456.482,550.670,3,1,2,2022,1,550.670,01001020803
...,...,...,...,...,...,...,...,...,...,...
9215,010010207003002,15.327,9.903,3,2,26,2022,4,19.806,01001020700
9216,010010207003031,440.002,21.072,2,1,37,2022,4,21.072,01001020700
9217,010010207003028,337.272,312.214,10,3,17,2022,4,936.642,01001020700
9218,010010205011028,337.272,312.214,10,3,17,2022,4,936.642,01001020501


In [100]:
df['census_tract'] = df['geoid'].str[:11]
avg_u_sum = df.groupby(['census_tract','year'])['upload_speed'].agg('sum').to_frame().reset_index()
devices_sum = df.groupby(['census_tract','year'])['devices'].agg('sum').to_frame().reset_index()
cdf = pd.merge(avg_u_sum, devices_sum, on=['census_tract','year'])
cdf['measure'] = 'avg_up_using_devices'
cdf['value'] = cdf['upload_speed']/cdf['devices']

# rename the census tract back into geoid to concat
cdf = cdf.rename(columns={'census_tract': 'geoid'})
cdf = cdf.reindex(STANDARD_COLS, axis="columns")
cdf['region_type'] = 'census_tract'
cdf

,geoid,measure,moe,value,year,region_type
0,01001020100,avg_up_using_devices,NaN,41.218120,2022,census_tract
1,01001020200,avg_up_using_devices,NaN,22.125842,2022,census_tract
2,01001020300,avg_up_using_devices,NaN,20.700021,2022,census_tract
3,01001020400,avg_up_using_devices,NaN,16.536312,2022,census_tract
4,01001020501,avg_up_using_devices,NaN,52.940173,2022,census_tract
5,01001020502,avg_up_using_devices,NaN,32.926654,2022,census_tract
6,01001020503,avg_up_using_devices,NaN,26.998774,2022,census_tract
7,01001020600,avg_up_using_devices,NaN,33.493672,2022,census_tract
8,01001020700,avg_up_using_devices,NaN,44.379046,2022,census_tract
9,01001020801,avg_up_using_devices,NaN,105.508076,2022,census_tract


In [99]:
df['county'] = df['geoid'].str[:5]
avg_u_sum = df.groupby(['county','year'])['upload_speed'].agg('sum').to_frame().reset_index()
devices_sum = df.groupby(['county','year'])['devices'].agg('sum').to_frame().reset_index()
codf = pd.merge(avg_u_sum, devices_sum, on=['county','year'])
codf['measure'] = 'avg_up_using_devices'
codf['value'] = codf['upload_speed']/codf['devices']

# rename the census tract back into geoid to concat
codf = codf.rename(columns={'county': 'geoid'})
codf = codf.reindex(STANDARD_COLS, axis="columns")
codf['region_type'] = 'county'
codf

,geoid,measure,moe,value,year,region_type
0,01001,avg_up_using_devices,NaN,88.039972,2022,county


In [103]:
final_df = pd.concat([codf, cdf, fdf])
final_df

,geoid,measure,moe,value,year,region_type
0,01001,avg_up_using_devices,NaN,88.039972,2022,county
0,01001020100,avg_up_using_devices,NaN,41.218120,2022,census_tract
1,01001020200,avg_up_using_devices,NaN,22.125842,2022,census_tract
2,01001020300,avg_up_using_devices,NaN,20.700021,2022,census_tract
3,01001020400,avg_up_using_devices,NaN,16.536312,2022,census_tract
...,...,...,...,...,...,...
1268,010010211004042,avg_up_using_devices,NaN,12.656625,2022,block_group
1269,010010211004043,avg_up_using_devices,NaN,11.617857,2022,block_group
1270,010010211004044,avg_up_using_devices,NaN,14.747615,2022,block_group
1271,010010211004045,avg_up_using_devices,NaN,11.617857,2022,block_group
